In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import sda_pseudo_grid_search
from keras.models import load_model


import os
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

#variables para guardar los resultados
tipo = pruebas[4]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Dataset de Amazon

In [3]:
print tipo
print dataset_name
print data_path
print dims

sda
amazon
data
1000


In [4]:
# cargando dataset de Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print X.shape

print "Todos los datos disponibles obtenidos"

(27677, 1000)
Todos los datos disponibles obtenidos


In [7]:
# se divide el dataset para los datos de entrenamiento y validacion del SDA
X_train, X_val, _, _ = train_test_split(X, np.zeros(X.shape[0]), test_size=0.2, random_state=42)

print X_train.shape
print X_val.shape

(22141, 1000)
(5536, 1000)


# Adaptación


## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/amazon/sda/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en una lista, la cual es almacenada en: models/amazon/sda/me1_models_paths.pkl

```python
paths_list = [{'autoencoder': ruta_autoencoder, 'encoder': ruta_encoder}, ..., {'autoencoder': ruta_autoencoder, 'encoder': ruta_encoder}]
```

In [20]:
models_paths = os.path.join(models_path, dataset_name, tipo, "me1_models_paths.pkl")
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'noises': [0.3 , 0.5, 0.8],
        'layers': [[int(dims/2)], [int(dims/2), int(dims/4)]],
        'epochs': [50, 25],
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_"
        
    paths_list = sda_pseudo_grid_search(X_train, X_val, parametros, folder_path, prefix)

    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en models/amazon/sda/me1_ae_0.h5
	Guardando encoder en models/amazon/sda/me1_e_0.h5
pr: 0.300 - epochs: 25 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en models/amazon/sda/me1_ae_1.h5
	Guardando encoder en models/amazon/sda/me1_e_1.h5
pr: 0.300 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...
	Guardando autoencoder en models/amazon/sda/me1_ae_2.h5
	Guardando encoder en models/amazon/sda/me1_e_2.h5
pr: 0.300 - epochs: 25 - layers: [500, 250]
	Entrenando autoencoder...
	Guardando autoencoder en models/amazon/sda/me1_ae_3.h5
	Guardando encoder en models/amazon/sda/me1_e_3.h5
pr: 0.500 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en models/amazon/sda/me1_ae_4.h5
	Guardando encoder en models/amazon/sda/me1_e_4.h5
pr: 0.500 - epochs: 25 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en 

In [21]:
print "Modelos almacenados en:"
for rutas in paths_list:
    print "\tautoencoder- %s\nencoder - %s\n" % (rutas['autoencoder'], rutas['encoder'])

Modelos almacenados en:
	autoencoder- models/amazon/sda/me1_ae_0.h5
encoder - models/amazon/sda/me1_e_0.h5

	autoencoder- models/amazon/sda/me1_ae_1.h5
encoder - models/amazon/sda/me1_e_1.h5

	autoencoder- models/amazon/sda/me1_ae_2.h5
encoder - models/amazon/sda/me1_e_2.h5

	autoencoder- models/amazon/sda/me1_ae_3.h5
encoder - models/amazon/sda/me1_e_3.h5

	autoencoder- models/amazon/sda/me1_ae_4.h5
encoder - models/amazon/sda/me1_e_4.h5

	autoencoder- models/amazon/sda/me1_ae_5.h5
encoder - models/amazon/sda/me1_e_5.h5

	autoencoder- models/amazon/sda/me1_ae_6.h5
encoder - models/amazon/sda/me1_e_6.h5

	autoencoder- models/amazon/sda/me1_ae_7.h5
encoder - models/amazon/sda/me1_e_7.h5

	autoencoder- models/amazon/sda/me1_ae_8.h5
encoder - models/amazon/sda/me1_e_8.h5

	autoencoder- models/amazon/sda/me1_ae_9.h5
encoder - models/amazon/sda/me1_e_9.h5

	autoencoder- models/amazon/sda/me1_ae_10.h5
encoder - models/amazon/sda/me1_e_10.h5

	autoencoder- models/amazon/sda/me1_ae_11.h5
encod

## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = {'autoencoder': ruta_autoencoder, 'encoder': ruta_encoder},
    .
    .
    .
    [dominio_n] = {'autoencoder': ruta_autoencoder, 'encoder': ruta_encoder},
}
```

Este diccionario queda almacenado en la ruta:
    models/amazon/sda/me1_best_models.pkl

In [23]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_best_models.pkl")
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for sda_model_path in paths_list:
            encoder = load_model(sda_model_path['encoder'])

            # se obtienen los datos del dominio
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            #se adapta el dominio segun el modelo
            X_deep = encoder.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "\t%d - score: %.4f" % (i, new_score)
            
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = sda_model_path['encoder']
            i = i+1

        #se guarda el mejor modelo para este dominio
        best_models[domain] = best_path

    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
kitchen


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/keras/models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your dat

1 - score: 0.7975


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your d

2 - score: 0.8050


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your d

3 - score: 0.7588


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your d

4 - score: 0.7450
5 - score: 0.7963
6 - score: 0.8075
7 - score: 0.7712
8 - score: 0.7481
9 - score: 0.8044
10 - score: 0.8019
11 - score: 0.7656
12 - score: 0.7525
dvd
1 - score: 0.7300
2 - score: 0.7519
3 - score: 0.7131
4 - score: 0.6863
5 - score: 0.7512
6 - score: 0.7275
7 - score: 0.6656
8 - score: 0.6944
9 - score: 0.7281
10 - score: 0.7394
11 - score: 0.7025
12 - score: 0.6881
electronics
1 - score: 0.8087
2 - score: 0.7894
3 - score: 0.7531
4 - score: 0.7619
5 - score: 0.8044
6 - score: 0.7937
7 - score: 0.7469
8 - score: 0.7288
9 - score: 0.7975
10 - score: 0.7969
11 - score: 0.7588
12 - score: 0.7506
books
1 - score: 0.7056
2 - score: 0.7200
3 - score: 0.6737
4 - score: 0.7006
5 - score: 0.7131
6 - score: 0.7081
7 - score: 0.6806
8 - score: 0.7019
9 - score: 0.7394
10 - score: 0.7094
11 - score: 0.7019
12 - score: 0.7000
Rutas guardadas en  models/amazon/sda/me1_best_models.pkl


In [24]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

dvd - models/amazon/sda/me1_e_1.h5
electronics - models/amazon/sda/me1_e_0.h5
books - models/amazon/sda/me1_e_8.h5
kitchen - models/amazon/sda/me1_e_5.h5


In [25]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        encoder = load_model(best_model_path)
        
        print "Adaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        
        tr_reps = encoder.predict(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps,
        }
        
        i = i+1
    else:
        print "Generar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para kitchen
Adaptando dominio
Cargando mejor modelo para dvd
Adaptando dominio
Cargando mejor modelo para electronics
Adaptando dominio
Cargando mejor modelo para books
Adaptando dominio
4/4 dominios adaptados


## Pruebas de clasificación

In [26]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    sda_adapter = load_model(ruta)
    
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #############
            #### SDA ####
            #############
            print "Adaptando dominios..."
            
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = sda_adapter.predict(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Entrenando clasificador adaptado.
Tarea 11 de 12
Cargando model

In [27]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,SDA,k->d,kitchen,dvd,26.25,41.50,15.25
2,SDA,k->e,kitchen,electronics,23.50,25.50,2.00
3,SDA,k->b,kitchen,books,25.50,39.75,14.25
4,SDA,d->k,dvd,kitchen,20.75,28.25,7.50
5,SDA,d->e,dvd,electronics,23.50,36.50,13.00
6,SDA,d->b,dvd,books,25.50,33.00,7.50
7,SDA,e->k,electronics,kitchen,20.75,21.50,0.75
8,SDA,e->d,electronics,dvd,26.25,31.25,5.00
9,SDA,e->b,electronics,books,25.50,39.75,14.25
10,SDA,b->k,books,kitchen,20.75,30.50,9.75


In [28]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/sda/me1_1000.csv
Resultados guardados.
